In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
import torch

# loading model and processor
model_id = "microsoft/Phi-3-vision-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map=None,
    trust_remote_code=True, 
    torch_dtype="auto",
    _attn_implementation='eager'  # deactivate Flash Attention cause of AMD GPU
)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)


In [ ]:
# defining the target prompt
messages = [ 
    {"role": "user", "content": "<|image_1|>\n...put your text prompt here..."} 
] 

# loading sample document image
image_path = r"put path to your image here..."
image = Image.open(image_path)

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# data preparation for the model
inputs = processor(prompt, [image], return_tensors="pt").to("cpu")  # set device to cpu

generation_args = {
    "max_new_tokens": 100,
    "temperature": 0.0,
    "do_sample": False,
}

# Generating text...
generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

# remove input-tokens
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(response)
